In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import text2emotion as te
import tkinter as tk

[nltk_data] Downloading package stopwords to /Users/Kenny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Kenny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Kenny/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
movie_df = pd.read_csv("data_adjs.csv")

In [3]:
# Combine all reviews' adjectives belonging to one same movie
movie_adj= movie_df.groupby('movieId').apply(lambda x: x['adjectives_merge'].values).to_frame(name="adj")
movie_adj.reset_index(inplace=True)

In [4]:
# Define the function to drop duplicate, merge list and drop stopwords.
def merge_list(list_):
    return ' '.join(list_)

def drop_duplicate(adj_list):
    return list(set(adj_list.split(' ')))

def drop_stop(_list):
    drop_list = ['much', 'first', 'many', 'non', 'human', 'last', 'main', 'sure', 'full', \
                 'anti', 'top', 'whole', 'little', 'able', 'true', 'hard', 'final', 'second', \
                 'entire','long', 'year', 'big', 'small', 'several', 'short','overall', 'next', \
                 'obvious', 'self', 'right', 'actual']
    stop = list(stopwords.words('english')) + drop_list
    
    return list(set(_list)-set(stop))

In [5]:
# Creating new column with targets we want using apply and function.
movie_adj['adjectives_merge'] = movie_adj['adj'].apply(merge_list)

movie_adj['adjectives_set'] = movie_adj['adjectives_merge'].apply(drop_duplicate)

movie_adj['adjectives_with_drop'] = movie_adj['adjectives_set'].apply(drop_stop)

movie_adj['adjectives_with_drop_merge'] = movie_adj['adjectives_with_drop'].apply(merge_list)

In [9]:
# Define the function of get emotion.
def get_emotion(sentence):
    e = te.get_emotion(sentence)
    return max(e, key = e.get)

In [10]:
# Calculate the sentiment.
movie_adj['label'] = movie_adj['adjectives_merge'].apply(get_emotion)
movie_adj['label'].value_counts()

Happy    222
Fear      22
Sad        6
Name: label, dtype: int64

In [11]:
# Read the movie name file and add it to our dataframe.
movie_list= pd.read_csv("top_movie_list.csv")

sorted_df = movie_list.sort_values(by=['movieId'])
sort_df = sorted_df.reset_index()
movie_adj['movieName'] = sort_df['movieName']

In [46]:
movie_adj.head()

,movieId,adj,adjectives_merge,adjectives_set,adjectives_with_drop,adjectives_with_drop_merge,label,movieName
0,12349,[great unsentimental latter soft-shoe physical...,great unsentimental latter soft-shoe physical ...,"[analogous, perfect, artistic, evident, hyster...","[analogous, hysterical, artistic, evident, per...",analogous hysterical artistic evident perfect ...,Happy,The Kid
1,15324,[surreal dazzling original perfect dream-like ...,surreal dazzling original perfect dream-like f...,"[incorrect, perfect, artistic, favourite, hyst...","[incorrect, perfect, artistic, favourite, hyst...",incorrect perfect artistic favourite hysterica...,Happy,Sherlock Jr.
2,15864,[famous hard young only tough memorable amusin...,famous hard young only tough memorable amusing...,"[vivid, unsuccessful, perfect, artistic, evide...","[vivid, unsuccessful, perfect, artistic, evide...",vivid unsuccessful perfect artistic evident im...,Happy,The Gold Rush
3,17136,[social underground dominant upper necessary o...,social underground dominant upper necessary ot...,"[structural, minute, indelible, incorrect, cou...","[minute, incorrect, couple, favourite, artisti...",minute incorrect couple favourite artistic liv...,Happy,Metropolis
4,17925,[straightforward clever good valuable sad unbe...,straightforward clever good valuable sad unbel...,"[minute, what-do-critics-know, perfect, brief,...","[minute, what-do-critics-know, favourite, perf...",minute what-do-critics-know favourite perfect ...,Happy,The General


In [44]:
# find most common adjs in all movies
k = []
for i in movie_adj['adj']:
    k+=list(i)

l = merge_list(k).split(' ')
j = []
for i in l:
    if i not in stop:
        j.append(i)
        
from collections import Counter
Counter(j).most_common(20)

[('great', 12701),
 ('good', 11508),
 ('real', 5145),
 ('old', 4017),
 ('young', 3548),
 ('new', 3425),
 ('bad', 3389),
 ('different', 3350),
 ('perfect', 3214),
 ('beautiful', 2765),
 ('classic', 2579),
 ('excellent', 2517),
 ('original', 2445),
 ('brilliant', 2284),
 ('amazing', 2251),
 ('wonderful', 2208),
 ('funny', 2078),
 ('interesting', 2076),
 ('special', 1999),
 ('American', 1979)]

In [13]:
movie_adj.to_csv('processed_data_adjs.csv')